In [1]:
#impor packages
import pandas as pd
import numpy as np
import os
import pickle
import json
import re

In [2]:
# location of raw data files
input_path = "../datasets/raw/"

In [3]:
#read LABEVENTS.csv file
df = pd.read_csv(filepath_or_buffer=input_path+'INPUTEVENTS_CV.csv',dtype = "object")

In [4]:
# Print datatype and non-null data points for each column
df.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17527935 entries, 0 to 17527934
Data columns (total 22 columns):
ROW_ID               17527935 non-null object
SUBJECT_ID           17527935 non-null object
HADM_ID              17505266 non-null object
ICUSTAY_ID           17497482 non-null object
CHARTTIME            17527935 non-null object
ITEMID               17527935 non-null object
AMOUNT               12422557 non-null object
AMOUNTUOM            13195400 non-null object
RATE                 4132022 non-null object
RATEUOM              4172256 non-null object
STORETIME            17527935 non-null object
CGID                 17173928 non-null object
ORDERID              17527935 non-null object
LINKORDERID          17527935 non-null object
STOPPED              480551 non-null object
NEWBOTTLE            245899 non-null object
ORIGINALAMOUNT       9972592 non-null object
ORIGINALAMOUNTUOM    13248996 non-null object
ORIGINALROUTE        17421252 non-null object
ORIGINALRATE      

In [5]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,ORDERID,LINKORDERID,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE
0,592,24457,184834,205776,2193-09-11 09:00:00,30056,100,ml,NaN,NaN,...,756654,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
1,593,24457,184834,205776,2193-09-11 12:00:00,30056,200,ml,NaN,NaN,...,3564075,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
2,594,24457,184834,205776,2193-09-11 16:00:00,30056,160,ml,NaN,NaN,...,422646,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
3,595,24457,184834,205776,2193-09-11 19:00:00,30056,240,ml,NaN,NaN,...,5137889,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
4,596,24457,184834,205776,2193-09-11 21:00:00,30056,50,ml,NaN,NaN,...,8343792,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN


In [6]:
#Dropping Row_ID 
df = df.drop(labels='ROW_ID',axis = 'columns')

In [7]:
#Dropping Row_ID 
df = df.drop(labels=['CGID','ORDERID','LINKORDERID','CHARTTIME','STOPPED','NEWBOTTLE'],axis = 'columns')

In [8]:
#read LABEVENTS.csv file
df1 = pd.read_csv(filepath_or_buffer=input_path+'D_ITEMS.csv',dtype = "object")

In [9]:
df1.head()

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
0,457,497,Patient controlled analgesia (PCA) [Inject],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1,458,498,PCA Lockout (Min),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2,459,499,PCA Medication,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3,460,500,PCA Total Dose,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4,461,501,PCV Exh Vt (Obser),NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [10]:
# bring LABEL name of ITEMID into the dataframe
df = df.merge(df1[['ITEMID','LABEL']], how='left', left_on='ITEMID', right_on='ITEMID')

In [11]:
df.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,STORETIME,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL
0,24457,184834,205776,30056,100,ml,NaN,NaN,2193-09-11 11:12:00,NaN,ml,Oral,NaN,NaN,NaN,Po Intake
1,24457,184834,205776,30056,200,ml,NaN,NaN,2193-09-11 12:05:00,NaN,ml,Oral,NaN,NaN,NaN,Po Intake
2,24457,184834,205776,30056,160,ml,NaN,NaN,2193-09-11 16:52:00,NaN,ml,Oral,NaN,NaN,NaN,Po Intake
3,24457,184834,205776,30056,240,ml,NaN,NaN,2193-09-11 19:06:00,NaN,ml,Oral,NaN,NaN,NaN,Po Intake
4,24457,184834,205776,30056,50,ml,NaN,NaN,2193-09-11 21:19:00,NaN,ml,Oral,NaN,NaN,NaN,Po Intake


In [12]:
#merge VALUE and UOM
df['AMOUNT_VALUE_UOM'] = df['ITEMID']+'-'+df['AMOUNT']+'-'+df['AMOUNTUOM'] 
df['RATE_VALUE_UOM'] = df['ITEMID']+'-'+df['RATE']+'-'+df['RATEUOM']

In [13]:
df = df.drop(labels=['AMOUNT','AMOUNTUOM','RATE','RATEUOM','ORIGINALAMOUNT','ORIGINALAMOUNTUOM','ORIGINALRATE','ORIGINALRATEUOM','ITEMID'], axis = 'columns')

In [18]:
df.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STORETIME,ORIGINALROUTE,ORIGINALSITE,LABEL,AMOUNT_VALUE_UOM,RATE_VALUE_UOM
0,24457,184834,205776,2193-09-11 11:12:00,Oral,NaN,Po Intake,30056-100-ml,NaN
1,24457,184834,205776,2193-09-11 12:05:00,Oral,NaN,Po Intake,30056-200-ml,NaN
2,24457,184834,205776,2193-09-11 16:52:00,Oral,NaN,Po Intake,30056-160-ml,NaN
3,24457,184834,205776,2193-09-11 19:06:00,Oral,NaN,Po Intake,30056-240-ml,NaN
4,24457,184834,205776,2193-09-11 21:19:00,Oral,NaN,Po Intake,30056-50-ml,NaN


In [22]:
#replace missingn value with NaN
df = df.fillna('NaN')

In [23]:
# create inputevents whitespace seperated string
df['inputevents_cv'] = df['LABEL']+' '+df['ORIGINALROUTE']+' '+df['ORIGINALSITE']+' '+df['AMOUNT_VALUE_UOM']+' '+df['RATE_VALUE_UOM']

In [28]:
# Drop redundant columns
df = df.drop(labels = ['ORIGINALROUTE','ORIGINALSITE','LABEL','AMOUNT_VALUE_UOM','RATE_VALUE_UOM'], axis = 'columns')

In [29]:
df.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STORETIME,inputevents_cv
0,24457,184834,205776,2193-09-11 11:12:00,Po Intake Oral NaN 30056-100-ml NaN
1,24457,184834,205776,2193-09-11 12:05:00,Po Intake Oral NaN 30056-200-ml NaN
2,24457,184834,205776,2193-09-11 16:52:00,Po Intake Oral NaN 30056-160-ml NaN
3,24457,184834,205776,2193-09-11 19:06:00,Po Intake Oral NaN 30056-240-ml NaN
4,24457,184834,205776,2193-09-11 21:19:00,Po Intake Oral NaN 30056-50-ml NaN


In [30]:
# Split the each event by whitespace
df['inputevents_cv'] = df['inputevents_cv'].map(lambda x: x.split()) 

In [31]:
# remove 'NaN' from all events
df['inputevents_cv'] = df['inputevents_cv'].map(lambda x: list(filter(lambda a: a!='NaN',x)))

In [32]:
df.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STORETIME,inputevents_cv
0,24457,184834,205776,2193-09-11 11:12:00,"[Po, Intake, Oral, 30056-100-ml]"
1,24457,184834,205776,2193-09-11 12:05:00,"[Po, Intake, Oral, 30056-200-ml]"
2,24457,184834,205776,2193-09-11 16:52:00,"[Po, Intake, Oral, 30056-160-ml]"
3,24457,184834,205776,2193-09-11 19:06:00,"[Po, Intake, Oral, 30056-240-ml]"
4,24457,184834,205776,2193-09-11 21:19:00,"[Po, Intake, Oral, 30056-50-ml]"


In [33]:
# create output path
mypath_output = "../datasets/inputevents_cv/tokenized/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [34]:
# export the dataframe to JSON format
df.to_json(mypath_output+"inputevents_cv.json",orient = 'records') 